# **Import Libraries**

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as ms


# **Load & Display Data**

In [ ]:
from google.colab import files
uploaded = files.upload()

flight_data = pd.read_csv('Flight Satisfation Tweets.csv')
flight_data.head()

Saving Flight Satisfation Tweets.csv to Flight Satisfation Tweets.csv


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


# **The Shape of Data**

In [ ]:
flight_data.shape

(14640, 15)

# **Display Information About Training Dataframe**

In [ ]:
flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

# **Convert Airline Sentiment Column to Numerical values**

In [ ]:
new_flight_data = flight_data.select_dtypes(include=['object']).copy()
new_flight_data.head()

,airline_sentiment,negativereason,airline,airline_sentiment_gold,name,negativereason_gold,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,neutral,NaN,Virgin America,NaN,cairdin,NaN,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,positive,NaN,Virgin America,NaN,jnardino,NaN,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,neutral,NaN,Virgin America,NaN,yvonnalynn,NaN,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,negative,Bad Flight,Virgin America,NaN,jnardino,NaN,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,negative,Can't Tell,Virgin America,NaN,jnardino,NaN,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
new_flight_data["airline_sentiment_code"] = np.where(new_flight_data["airline_sentiment"].str.contains("positive"), 1, 0)
new_flight_data

,airline_sentiment,negativereason,airline,airline_sentiment_gold,name,negativereason_gold,text,tweet_coord,tweet_created,tweet_location,user_timezone,airline_sentiment_code
0,neutral,NaN,Virgin America,NaN,cairdin,NaN,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),0
1,positive,NaN,Virgin America,NaN,jnardino,NaN,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),1
2,neutral,NaN,Virgin America,NaN,yvonnalynn,NaN,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),0
3,negative,Bad Flight,Virgin America,NaN,jnardino,NaN,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0
4,negative,Can't Tell,Virgin America,NaN,jnardino,NaN,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0
...,...,...,...,...,...,...,...,...,...,...,...,...
14635,positive,NaN,American,NaN,KristenReenders,NaN,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN,1
14636,negative,Customer Service Issue,American,NaN,itsropes,NaN,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN,0
14637,neutral,NaN,American,NaN,sanyabun,NaN,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN,0
14638,negative,Customer Service Issue,American,NaN,SraJackson,NaN,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada),0


# **Value Count on the Feature name Label**

In [ ]:
new_flight_data['airline_sentiment_code'].value_counts()

0    12277
1     2363
Name: airline_sentiment_code, dtype: int64

# **Data Preprocessing**

In [ ]:
#cleaning the data

#def drop_features(features,data):
    #data.drop(features,inplace=True,axis=1)

# **Remove Some Characters in the Data**

In [ ]:
import re
## example ## 
re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ","ouch...junior is angryð#got7 #junior #yugyo..., @user")

'ouch   junior is angry     got7  junior  yugyo      '

In [ ]:
def process_text(text):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",text.lower()).split())

In [ ]:
new_flight_data['text'] = new_flight_data['text'].apply(process_text)

In [ ]:
new_flight_data.head(10)

,airline_sentiment,negativereason,airline,airline_sentiment_gold,name,negativereason_gold,text,tweet_coord,tweet_created,tweet_location,user_timezone,airline_sentiment_code
0,neutral,NaN,Virgin America,NaN,cairdin,NaN,what said,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),0
1,positive,NaN,Virgin America,NaN,jnardino,NaN,plus you ve added commercials to the experienc...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),1
2,neutral,NaN,Virgin America,NaN,yvonnalynn,NaN,i didn t today must mean i need to take anothe...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),0
3,negative,Bad Flight,Virgin America,NaN,jnardino,NaN,it s really aggressive to blast obnoxious ente...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0
4,negative,Can't Tell,Virgin America,NaN,jnardino,NaN,and it s a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0
5,negative,Can't Tell,Virgin America,NaN,jnardino,NaN,seriously would pay 30 a flight for seats that...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada),0
6,positive,NaN,Virgin America,NaN,cjmcginnis,NaN,yes nearly every time i fly vx this ear worm w...,NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada),1
7,neutral,NaN,Virgin America,NaN,pilot,NaN,really missed a prime opportunity for men with...,NaN,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada),0
8,positive,NaN,Virgin America,NaN,dhepburn,NaN,well i didn t but now i do d,NaN,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada),1
9,positive,NaN,Virgin America,NaN,YupitsTate,NaN,it was amazing and arrived an hour early you r...,NaN,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada),1


# **Remove Features id & tweet from Dataframe**

In [ ]:
new_flight_data_columns = new_flight_data[["airline_sentiment", "text", "airline_sentiment_code"]]
new_flight_data_columns


,airline_sentiment,text,airline_sentiment_code
0,neutral,what said,0
1,positive,plus you ve added commercials to the experienc...,1
2,neutral,i didn t today must mean i need to take anothe...,0
3,negative,it s really aggressive to blast obnoxious ente...,0
4,negative,and it s a really big bad thing about it,0
...,...,...,...
14635,positive,thank you we got on a different flight to chicago,1
14636,negative,leaving over 20 minutes late flight no warning...,0
14637,neutral,please bring american airlines to blackberry10,0
14638,negative,you have my money you change my flight and don...,0


In [ ]:
new_flight_data_columns.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   airline_sentiment       14640 non-null  object
 1   text                    14640 non-null  object
 2   airline_sentiment_code  14640 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 343.2+ KB


# **Import Relevant Lib to split Dataframe into Train & Test**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new_flight_data_columns["text"], new_flight_data_columns["airline_sentiment_code"], test_size = 0.2, random_state = 65)


# **Transform Data & Put in Vectors**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

In [ ]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

In [ ]:
print(x_train_counts.shape)
print(x_train_tfidf.shape)

(11712, 12348)
(11712, 12348)


In [ ]:
x_train_counts

<11712x12348 sparse matrix of type '<class 'numpy.int64'>'
	with 94367 stored elements in Compressed Sparse Row format>

In [ ]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

In [ ]:
print(x_test_counts.shape)
print(x_test_tfidf.shape)

(2928, 12348)
(2928, 12348)


In [ ]:
x_test_counts

<2928x12348 sparse matrix of type '<class 'numpy.int64'>'
	with 21998 stored elements in Compressed Sparse Row format>

# **Model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)
model.fit(x_train_tfidf,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
predictions = model.predict(x_test_tfidf)


In [ ]:
predict_train = model.predict(x_train_tfidf)

# **Confusion Matrix (Train Data)**

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_train,predict_train)

array([[9781,   25],
       [  20, 1886]])

In [ ]:
tp= 9781
tn = 1886
fn = 20
fp = 25

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = 2 * (recall * precision) / (recall + precision)
f1_score

0.9977049013107564

In [ ]:
predict_train

array([0, 0, 0, ..., 0, 0, 1])

# **Confusion Matrix (Test Data)**

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,predictions)

array([[2383,   88],
       [ 219,  238]])

In [ ]:
tp= 2383
tn = 243
fn = 214
fp = 88

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = 2 * (recall * precision) / (recall + precision)
f1_score

0.9404104183109707

In [ ]:
predictions

array([0, 0, 1, ..., 0, 0, 0])

# **Present Final Result**

In [ ]:
final_result = pd.DataFrame({'text':x_train,'sentiments':predict_train})

In [ ]:
final_result.head(30)

,text,sentiments
6068,flt 463 san jose ca gt denver co delayed 2 hrs...,0
12515,thanks for the generic computer generated resp...,0
3708,please help trace my luggage which was put thr...,0
11330,ya ll need to work on your online checkout sys...,0
10501,to a booked hotel for the night so i had to fi...,0
7321,ooookay that s silly of you guess we re watchi...,0
9822,it s flt 4827 to elm still no gate and still h...,0
12494,just went to check into flight 24 hr advance a...,0
6468,bos to msp msp to aus aus to bos site doesn t ...,0
12184,you could train your flight attendants to have...,0
